In [4]:
import random
import numpy as np
np.random.seed(seed=1337)
max_weight = 10
n_items = 32
weights =  np.random.rand(n_items)
values = np.random.rand(n_items)

In [13]:
def encodeNumberHigh(bits):
    def f(number):
        return [number & 2**x != 0 for x in range(n_items-1,-1,-1)]
    return f
encodeNumber = encodeNumberHigh(n_items)

In [25]:
def scoreFunction(mask, weights=weights,values=values,max_weight=max_weight):
    return 0 if max_weight<np.sum(weights[mask]) else np.sum(values[mask])

0.9973891432208869